# Model Training & Data Org Functions
### Begin with loading data & libs:

In [13]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import normalize
BATCH_SIZE=2
# The following lines adjust the granularity of reporting. 
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

In [14]:
trainset_01 = pd.read_csv(filepath_or_buffer="./trainset.csv")

Let's get some basic info on the data:

In [15]:
#trainset_01.head()
#trainset_01.describe()
#trainset_01.corr()

### Function Definitions:

In [16]:
def get_fbank(item):
    a = np.load(item)
    a = np.resize(a, (2900,10,1))
    return a
def load_data(input_file):
    dfData = pd.read_csv(input_file)
    dfLabels = dfData['Subject'].astype('category')
    dfFiles = dfData['FileName']
    le = LabelEncoder()
    targets = le.fit_transform(dfLabels)
    inputs = []
    for i in range(0, dfFiles.shape[0]):
        inputs.append(get_fbank(dfFiles.at[i]))
    inputs = np.stack(inputs)
    #xmin = inputs.min()
    #xmax = inputs.max()
    #inputs = (inputs-xmin)/(xmax-xmin)
    return inputs, tf.keras.utils.to_categorical(targets), len(np.unique(targets))
def plot_the_loss_curve(epochs, rmse):
  """Plot a curve of loss vs. epoch."""
  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Root Mean Squared Error")
  plt.plot(epochs, rmse, label="Loss")
  plt.legend()
  plt.ylim([rmse.min()*0.94, rmse.max()* 1.05])
  plt.show()  
def create_model(my_learning_rate, feature_layer):
  """Create and compile a simple linear regression model."""
  # Most simple tf.keras models are sequential.
  model = tf.keras.models.Model(inputs=ilayer, outputs=olayer)
  # Add the layer containing the feature columns to the model.
  model.add(feature_layer)
  # Add one linear layer to the model to yield a simple linear regressor.
  model.add(tf.keras.layers.Dense(units=1, input_shape=(1,)))
  # Construct the layers into a model that TensorFlow can execute.
  model.compile(optimizer="Adam",
                loss="categorical_crossentropy",
                metrics=[tf.keras.metrics.RootMeanSquaredError()])
  return model
def train_model(model, dataset, epochs, batch_size, label_name):
  """Feed a dataset into the model in order to train it."""
  features = {name:np.array(value) for name, value in dataset.items()}
  label = np.array(features.pop(label_name))
  history = model.fit(x=features, y=label, batch_size=batch_size,
                      epochs=epochs, shuffle=True)
  # The list of epochs is stored separately from the rest of history.
  epochs = history.epoch
  # Isolate the mean absolute error for each epoch.
  hist = pd.DataFrame(history.history)
  rmse = hist["root_mean_squared_error"]
  return epochs, rmse

In [17]:
# Load the data
inputs, targets, categories = load_data('./trainset.csv')

In [18]:
# Create the model
ilayer = tf.keras.layers.Input(shape=(2900,10,1), name="input")
x = tf.keras.layers.Conv2D(8, (3,3))(ilayer)
x = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2)(x)
x = tf.keras.layers.Conv2D(16, (3,3))(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2,1), strides=2)(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128, activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
olayer = tf.keras.layers.Dense(categories, activation="softmax", name="output")(x)
model = tf.keras.models.Model(inputs=ilayer, outputs=olayer)
model.summary()

model.compile(optimizer="Adam", loss="categorical_crossentropy")
model.fit(inputs, targets, epochs=10, batch_size=1, verbose=1)

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 2900, 10, 1)]     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 2898, 8, 8)        80        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1449, 4, 8)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1447, 2, 16)       1168      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 723, 1, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 11568)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)              